In [1]:
import sys
sys.path.insert(0, '/home/watts/Software/xgboost/python-package') # for xgboost

In [16]:
import re
import numpy as np
import pandas as pd
import feather
import xgboost as xgb
import feather

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

import pickle


In [3]:
def save_classifier(fname, clf):
    # save the classifier
    with open(fname, 'wb') as fid:
        pickle.dump(clf, fid)

def load_classifier(fname):
    # load it again
    with open(fname, 'rb') as fid:
        clf = pickle.load(fid)
        return clf

In [4]:
df_train = np.load('../cache/train_stage2_fe2.npy')

df1 = pd.read_csv('../cache/stage2_labels.csv')
y = df1['y'].values

df_test = np.load('../cache/test_stage2_fe2.npy')
df = pd.read_csv('../cache/stage2_test_id.csv')
pid = df.ID.values

In [5]:
df_train.shape

(22044, 3318)

In [6]:
print(y)

[1 2 2 ..., 6 4 1]


In [7]:
y = y - 1 #fix for zero bound array

In [8]:
print(y.shape)

(22044,)


In [9]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [10]:
wts = np.load('../cache/stage2_train_weights.npy')

In [11]:
# wts_per_class = np.load('../cache/stage2_train_weights_per_class.npy')
# wts_per_class = wts_per_class.tolist()

In [12]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob', sample_weight=wts)
xgbParams = xgb_model.get_xgb_params()
xgbParams['num_class'] = 9
parameters = {
    'learning_rate': [0.03, 0.035, 0.05], #so called `eta` value
    'max_depth': [5,6,10],
    'min_child_weight': [1,5,10],
    'silent': [1],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.8, 0.7],
    'missing':[-999],
    'seed': [1337]
}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, 5),
                   scoring = 'neg_log_loss',
                   verbose=3, refit=True)


# neg_log_loos:-1.336173588372656
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1,
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.77967
# 1.71664

In [17]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob', sample_weight=wts)
xgbParams = xgb_model.get_xgb_params()
xgbParams['num_class'] = 9
parameters = {
    'learning_rate': [0.03, 0.035, 0.05], #so called `eta` value
    'max_depth': [5,6,10],
    'min_child_weight': [1,5,10],
    'silent': [1],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.8, 0.7],
    'missing':[-999],
    'seed': [1337]
}


clf = RandomizedSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, 5, shuffle=True),
                   scoring = 'neg_log_loss',
                   verbose=3, refit=True)


In [18]:
clf.fit(df_train, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7 
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7 
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7 
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7 
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7 
[CV]  subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0.7, score=-1.461876 -23.0min
[CV] subsample=0.7, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=5, learning_rate=0.035, colsample_bytree=0

[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed: 155.2min


[CV]  subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=10, learning_rate=0.05, colsample_bytree=0.7, score=-1.388722 -38.7min
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=1, max_depth=6, learning_rate=0.05, colsample_bytree=0.7 
[CV]  subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=10, learning_rate=0.05, colsample_bytree=0.7, score=-1.529068 -38.1min
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=1, max_depth=6, learning_rate=0.05, colsample_bytree=0.7 
[CV]  subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=5, max_depth=10, learning_rate=0.05, colsample_bytree=0.7, score=-1.422893 -38.9min
[CV] subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=1, max_depth=6, learning_rate=0.05, colsample_bytree=0.7 
[CV]  subsample=0.8, silent=1, seed=1337, missing=-999, min_child_weight=1, max_depth=6, learning_rate=0.05, colsample_bytree=0.7, score=-1

[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed: 289.2min finished


RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 1 ..., 3 0], n_folds=5, shuffle=False, random_state=None),
          error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1,
       sample_weight=array([ 5.14381,  7.95733, ...,  2.7055 ,  5.14381]),
       scale_pos_weight=1, seed=None, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=10, n_jobs=5,
          param_distributions={'learning_rate': [0.03, 0.035, 0.05], 'max_depth': [5, 6, 10], 'min_child_weight': [1, 5, 10], 'silent': [1], 'subsample': [0.8, 0.7], 'colsample_bytree': [0.8, 0.7], 'missing': [-999], 'seed': [1337]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True

In [20]:
print(clf.best_score_)
print(clf.best_params_)

# f1_macro
# 0.4293248036851208
# {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.74021, 1.72588

# neg_log_loos:-1.336173588372656
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1,
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.77967
# 1.71664

# -1.3876712019506041 (whien using wts, use xgboost from github)
# {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 
#  'min_child_weight': 5, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# private LB: 2.44337, pub LB: 1.69402

# -1.4902400080711418
# {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 6, 'min_child_weight': 1, 
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.7}
# private LB 2.56616
# public LB: 1.71541

# pip xgboost -1.4811547760008215
# {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 5, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# In [14]:

# accuracy
# 0.5538086202222825
# {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# In [26]:

-1.5434049390529498
{'subsample': 0.8, 'silent': 1, 'seed': 1337, 'missing': -999, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.035, 'colsample_bytree': 0.7}


In [21]:
# print(clf.best_score_)
# print(clf.best_params_)
#best score -1.42 (  grid with wts prob not used)

In [22]:
# print(clf.best_score_)
# print(clf.best_params_)
# best score: -1.40 (no wts)

In [23]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
# print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))



colsample_bytree: 0.7
learning_rate: 0.035
max_depth: 5
min_child_weight: 5
missing: -999
seed: 1337
silent: 1
subsample: 0.8


In [24]:
test_probs = clf.predict_proba(df_test)

In [25]:
test_probs.shape

(986, 9)

In [26]:
print(test_probs)

[[ 0.1815663   0.09524849  0.05592067 ...,  0.12767667  0.03564896
   0.02722346]
 [ 0.80430394  0.04496374  0.00862814 ...,  0.0697874   0.00865818
   0.00714359]
 [ 0.08042137  0.32906529  0.03103557 ...,  0.33890304  0.03268041
   0.0191015 ]
 ..., 
 [ 0.0829633   0.35893023  0.0234056  ...,  0.31453186  0.01687983
   0.01648173]
 [ 0.34296426  0.1417527   0.04408211 ...,  0.14183071  0.02599195
   0.02207162]
 [ 0.11487987  0.24232443  0.03359992 ...,  0.30171379  0.03856931
   0.02482533]]


In [27]:
test_probs = np.clip(test_probs, a_min=0.05, a_max=0.95)

In [29]:
submission = pd.DataFrame(test_probs, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
# submission.to_csv('../submissions/subm_xgb_stage2_with_gridcv.csv', index=False)
# scored 1.70423 on the public leaderboard, 2.40243 on the private leaderboard


submission.to_csv('../submissions/subm_xgb_stage2_with_wt_gridcv.csv', index=False)
# scored 1.36438 on the public leaderboard, 2.35876 on the private leaderboard
